In [56]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn import metrics
import optuna
from sklearn.svm import SVC
from sklearn.utils import resample
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score


In [2]:
#carregar dataset
df = pd.read_csv('factnews_dataset.csv')
df.isnull().sum()

file          0
id_sente      0
id_article    0
domain        0
year          0
sentences     0
classe        0
dtype: int64

# Pré-processamento

divisão dos dados

In [4]:
'''
#criar grupo treino, validação e teste
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df['classe'])
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['classe'])

#guardar conjuntos em memória
train_df.to_csv("train.csv", index=False) 
val_df.to_csv("val.csv", index=False)
test_df.to_csv("test.csv", index=False)
'''

'\n#criar grupo treino, validação e teste\ntrain_df, test_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df[\'classe\'])\ntrain_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df[\'classe\'])\n\n#guardar conjuntos em memória\ntrain_df.to_csv("train.csv", index=False) \nval_df.to_csv("val.csv", index=False)\ntest_df.to_csv("test.csv", index=False)\n'

In [ ]:
train= pd.read_csv('train.csv')
val= pd.read_csv('val.csv')
test= pd.read_csv('test.csv')

train_oversampling= pd.read_csv('train_oversampling.csv')
val_oversampling= pd.read_csv('val_oversampling.csv')
test_oversampling= pd.read_csv('test_oversampling.csv')

y_train= train['classe']
y_train_oversampling= train_oversampling['classe']

y_val=val['classe']
y_val_oversampling = val_oversampling['classe']

y_test= test['classe']
y_test_oversampling = test_oversampling['classe']

Pré-processamento da primeira meta 

In [ ]:
df = pd.read_csv('factnews_dataset.csv')
def pre_processamento_meta1(df):
    stop_words = set(stopwords.words('portuguese'))
    #criar coluna tokens
    df['tokens'] = df['sentences'].apply(lambda x: nltk.word_tokenize(str(x).lower()))
    # Tokenizar sem stopwords
    df['tokens'] = df['tokens'].apply(lambda toks: [t for t in toks if t not in stop_words])
    return df

train_meta1 = pre_processamento_meta1(train)
val_meta1 = pre_processamento_meta1(val)
test_meta1= pre_processamento_meta1(test)

y_train_meta1=train_meta1['classe']
y_val_meta1=val_meta1['classe']
y_test_meta1=test_meta1['classe']

balanceamento dos dados

In [5]:
train= pd.read_csv('train.csv')
val= pd.read_csv('val.csv')
test= pd.read_csv('test.csv')

train_oversampling= pd.read_csv('train_oversampling.csv')
val_oversampling= pd.read_csv('val_oversampling.csv')
test_oversampling= pd.read_csv('test_oversampling.csv')

y_train= train['classe']
y_train_oversampling= train_oversampling['classe']

y_val=val['classe']
y_val_oversampling = val_oversampling['classe']

y_test= test['classe']
y_test_oversampling = test_oversampling['classe']

In [6]:
# Contagem das classes em cada conjunto
print ("conjunto de treino: ",y_train.value_counts().sort_index())
print("\nconjunto de validação: ",y_val.value_counts().sort_index())
print("\nconjunto de teste: ", y_test.value_counts().sort_index())
print("\nconjunto de treino + exemplos: ", y_train_oversampling.value_counts().sort_index())
print("\nconjunto de val + exemplos: ", y_val_oversampling.value_counts().sort_index())
print("\nconjunto de teste + exemplos: ", y_test_oversampling.value_counts().sort_index())



conjunto de treino:  classe
-1     779
 0    2375
 1     312
Name: count, dtype: int64

conjunto de validação:  classe
-1    195
 0    594
 1     78
Name: count, dtype: int64

conjunto de teste:  classe
-1     417
 0    1273
 1     168
Name: count, dtype: int64

conjunto de treino + exemplos:  classe
-1    2375
 0    2375
 1    2375
Name: count, dtype: int64

conjunto de val + exemplos:  classe
-1    594
 0    594
 1    594
Name: count, dtype: int64

conjunto de teste + exemplos:  classe
-1    1273
 0    1273
 1    1273
Name: count, dtype: int64


In [7]:
from sklearn.utils import resample

def balancear_classes(conjunto):
  # Separar cada classe
  class_min = conjunto[conjunto['classe'] == -1]
  class_med = conjunto[conjunto['classe'] == 1]
  class_max = conjunto[conjunto['classe'] == 0]

  # Número da classe maior
  n_samples = len(class_max)

  # Replicar as classes menores até o tamanho da maior
  class_min_upsampled = resample(class_min, 
                              replace=True, 
                              n_samples=n_samples, 
                              random_state=42)

  class_med_upsampled = resample(class_med, 
                              replace=True, 
                              n_samples=n_samples, 
                            random_state=42)

  conjunto_oversampled = pd.concat([class_min_upsampled, class_med_upsampled, class_max])

  # baralhar o conjunto 
  conjunto_oversampled = conjunto_oversampled.sample(frac=1, random_state=42).reset_index(drop=True)

  print(conjunto_oversampled['classe'].value_counts())

  return conjunto_oversampled

#train_oversampling.to_csv('train_oversampling.csv', index=False)
#val_oversampling = balancear_classes(val)
#test_oversampling= balancear_classes(test)

#val_oversampling.to_csv('val_oversampling.csv', index=False)
#test_oversampling.to_csv('test_oversampling.csv', index=False)


tokeinização

In [8]:
stop_words = set(stopwords.words('portuguese'))
def tokeinizar(train, val, test):

    #criar coluna tokens
    train['tokens'] = train['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
    val['tokens']   = val['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
    test['tokens'] = test['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))
    train_oversampling['tokens'] = train_oversampling['sentences'].apply(lambda x: nltk.word_tokenize(str(x)))

    #remover stopwords
    train['tokens'] = train['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])
    val['tokens']   = val['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])
    test['tokens']  = test['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])
    train_oversampling['tokens'] = train_oversampling['tokens'].apply(lambda toks: [t for t in toks if t.lower() not in stop_words])

    #matizar
    #nltk . download ('rslp')
    stemmer = nltk . stem . RSLPStemmer ()
    train['stems'] = train['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
    val['stems']   = val['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
    test['stems'] = test['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])
    train_oversampling['stems'] = train_oversampling['tokens'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

    nltk . download ('rslp')
    stemmer = nltk . stem . RSLPStemmer ()
    print(train['stems'].head())

    return train, val, test

train, val, test = tokeinizar(train, val, test)
train_oversampling, val_oversampling, test_oversampling = tokeinizar(train_oversampling, val_oversampling, test_oversampling)

[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\didia\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


0    [neym, ,, part, 20, novembr, vai, lider, seleç...
1    [forç, internac, transform, quest, taiwan, ass...
2    [alianç, mai, quant, parlament, eleit, nome, p...
3    [ger, mort, '', ,, declar, cabr, ,, 16ª, entre...
4    [bisp, barr, (, ba, ), ,, dom, luiz, flávi, ca...
Name: stems, dtype: object
0    [qued, ocorr, aliment, pux, cord, sent, contr,...
1    [repercuss, nega, cas, fez, bolsonar, ,, candi...
2    [brasil, lav, alm, após, decepcion, empat, col...
3    [final, encontr, ,, sen, tucan, diss, esp, gov...
4    [fiz, compromiss, líd, bas, hoj, líd, opos, ,,...
Name: stems, dtype: object


[nltk_data] Downloading package rslp to
[nltk_data]     C:\Users\didia\AppData\Roaming\nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [9]:
y_train= train['classe']
y_train_oversampling= train_oversampling['classe']

y_val=val['classe']
y_val_oversampling = val_oversampling['classe']

y_test= test['classe']
y_test_oversampling = test_oversampling['classe']

# Construção dos modelos de representação

## TF-IDF - fred

In [10]:
def TF_IDF(train, val, test):
    tf_vect = TfidfVectorizer(ngram_range =(1, 2), min_df =3, max_df =0.5, max_features =500)

        #juntar os tokens matizados em frases novvamente 
    text_train = train['stems'].apply(lambda tokens: ' '.join(tokens))
    text_val   = val['stems'].apply(lambda tokens: ' '.join(tokens))
    text_test = test['stems'].apply(lambda tokens: ' '.join(tokens))

    X_train = tf_vect.fit_transform(text_train)
    X_val   = tf_vect.transform(text_val)
    X_test  = tf_vect.transform(text_test)
    return X_train, X_val, X_test

In [ ]:
X_train_tfidf_vec, X_val_tfidf_vec, X_test_tfidf_vec = TF_IDF(train,val, test)
X_train_oversampling_tfidf_vec, X_val_oversampling_tfidf_vec, X_test_oversampling_tfidf_vec = TF_IDF(train_oversampling, val_oversampling, test_oversampling)

X_train_meta1_tfidf, X_val_meta1_tfidf, X_test_meta1_tfidf = TF_IDF(train_meta1,val_meta1, test_meta1)

## CountVectorizer -didi

In [12]:
import optuna
optuna.logging.set_verbosity(optuna.logging.ERROR)

In [13]:
import optuna
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

def objective_count_vec(trial, train, val):
    # Parâmetros a otimizar
    max_features = trial.suggest_int("max_features", 500, 5000)
    ngram_min = trial.suggest_int("ngram_min", 1, 2)
    ngram_max = trial.suggest_int("ngram_max", ngram_min, 3)  # garante ngram_max >= ngram_min

    # CountVectorizer com parâmetros sugeridos
    c_vect = CountVectorizer(
        token_pattern=r"(?u)\b\w+\b|[^\w\s]",
        ngram_range=(ngram_min, ngram_max),
        strip_accents='unicode',
        max_features=max_features
    )

    # Transformar textos
    text_train = train['stems'].apply(lambda tokens: ' '.join(tokens))
    text_val   = val['stems'].apply(lambda tokens: ' '.join(tokens))

    X_train = c_vect.fit_transform(text_train)
    X_val   = c_vect.transform(text_val)

    y_train = train['classe'] 
    y_val   = val['classe']

    # Treinar Logistic Regression
    model = LogisticRegression(max_iter=500)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)

    # Métrica: F1 macro
    score = f1_score(y_val, preds, average='macro')
    return score 

def count_vec(train, val, test):

    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_count_vec(trial, train,val),
                   n_trials=30, n_jobs=-1, show_progress_bar=False)
    
    best_params=study.best_params
    print("Melhores parâmetros:", study.best_params)
    print("Melhor F1:", study.best_value)
    
    c_vect = CountVectorizer(
        token_pattern=r"(?u)\b\w+\b|[^\w\s]",
        ngram_range=(best_params['ngram_min'], best_params['ngram_max']),
        strip_accents='unicode',
        max_features=best_params['max_features']
    )

    # Transformar textos
    text_train = train['stems'].apply(lambda tokens: ' '.join(tokens))
    text_val = val['stems'].apply(lambda tokens: ' '.join(tokens))
    text_test = test['stems'].apply(lambda tokens: ' '.join(tokens))

    # Fit no treino, transform nos outros conjuntos
    X_train = c_vect.fit_transform(text_train)
    X_val = c_vect.transform(text_val)
    X_test = c_vect.transform(text_test)

    return X_train, X_val, X_test

In [ ]:
X_train_count_vec, X_val_count_vec, X_test_count_vec = count_vec(train, val, test)
X_train_oversampling_count_vec, X_val_oversampling_count_vec, X_test_oversampling_count_vec= count_vec(train_oversampling, val_oversampling, test_oversampling)

X_train_meta1_count_vec, X_val_meta1_count_vec, X_test_meta1_count_vec = count_vec(train_meta1,val_meta1, test_meta1)

Melhores parâmetros: {'max_features': 3234, 'ngram_min': 1, 'ngram_max': 3}
Melhor F1: 0.680755783275675
Melhores parâmetros: {'max_features': 3738, 'ngram_min': 1, 'ngram_max': 1}
Melhor F1: 0.6950529365885298


## Glove -fred

In [15]:
import optuna
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

def objective_tfidf(trial):
    # Parâmetros a otimizar
    max_features = trial.suggest_int("max_features", 500, 5000)
    ngram_min = trial.suggest_int("ngram_min", 1, 2)
    ngram_max = trial.suggest_int("ngram_max", ngram_min, 3) 
    min_df = trial.suggest_int("min_df", 1, 10)
    max_df = trial.suggest_float("max_df", 0.3, 0.9)

    tfidf_vect = TfidfVectorizer(
        ngram_range=(ngram_min, ngram_max),
        min_df=min_df,
        max_df=max_df,
        max_features=max_features,
        strip_accents='unicode',
        token_pattern=r"(?u)\b\w+\b|[^\w\s]"
    )

    text_train = train['stems'].apply(lambda tokens: ' '.join(tokens))
    text_val = val['stems'].apply(lambda tokens: ' '.join(tokens))

    X_train = tfidf_vect.fit_transform(text_train)
    X_val = tfidf_vect.transform(text_val)

    y_train = train['classe']
    y_val = val['classe']

    model = LogisticRegression(max_iter=500)
    model.fit(X_train, y_train)
    preds = model.predict(X_val)

    score = f1_score(y_val, preds, average='macro')
    return score

def tfidf_optuna(train, val, test):
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_tfidf, n_trials=30)

    best_params = study.best_params
    print("Melhores parâmetros:", best_params)

    tfidf_vect = TfidfVectorizer(
        ngram_range=(best_params['ngram_min'], best_params['ngram_max']),
        min_df=best_params['min_df'],
        max_df=best_params['max_df'],
        max_features=best_params['max_features'],
        strip_accents='unicode',
        token_pattern=r"(?u)\b\w+\b|[^\w\s]"
    )

    text_train = train['stems'].apply(lambda tokens: ' '.join(tokens))
    text_val = val['stems'].apply(lambda tokens: ' '.join(tokens))
    text_test = test['stems'].apply(lambda tokens: ' '.join(tokens))

    X_train = tfidf_vect.fit_transform(text_train)
    X_val = tfidf_vect.transform(text_val)
    X_test = tfidf_vect.transform(text_test)

    return X_train, X_val, X_test


In [16]:
X_train_tfidf_vec, X_val_tfidf_vec, X_test_tfidf_vec = tfidf_optuna(train,val, test)
X_train_oversampling_tfidf_vec, X_val_oversampling_tfidf_vec, X_test_oversampling_tfidf_vec = tfidf_optuna(train_oversampling, val_oversampling, test_oversampling)

Melhores parâmetros: {'max_features': 526, 'ngram_min': 1, 'ngram_max': 1, 'min_df': 5, 'max_df': 0.553539201239897}
Melhores parâmetros: {'max_features': 731, 'ngram_min': 1, 'ngram_max': 1, 'min_df': 4, 'max_df': 0.5697303395749046}


## WORD2VEC -anaana

fontes : 
-   https://sites.google.com/view/nilc-usp/resources-and-tools?authuser=0
-   https://huggingface.co/nilc-nlp/word2vec-skip-gram-300d

In [17]:
#%pip install huggingface_hub
#%pip install hf_hub_download
#%pip install safetensors
from huggingface_hub import hf_hub_download
from safetensors.numpy import load_file

def carregar_word2vec():
    path = hf_hub_download(repo_id="nilc-nlp/word2vec-skip-gram-300d",
                           filename="embeddings.safetensors")

    data = load_file(path)
    vectors = data["embeddings"]

    vocab_path = hf_hub_download(repo_id="nilc-nlp/word2vec-skip-gram-300d",
                                 filename="vocab.txt")
    with open(vocab_path) as f:
        vocab = [w.strip() for w in f]

    print(vectors.shape)

    model = KeyedVectors(vector_size=vectors.shape[1])
    model.add_vectors(vocab, vectors)
    model.fill_norms()

    return model

def vetor_frase(model, frase):
    vectors = [model[w] for w in frase if w in model]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

def word2vec(train, val, test, coluna, model):
    X_train = np.vstack([vetor_frase(model, tokens) for tokens in train[coluna]])
    X_val   = np.vstack([vetor_frase(model, tokens) for tokens in val[coluna]])
    X_test  = np.vstack([vetor_frase(model, tokens) for tokens in test[coluna]])

    return X_train, X_val, X_test

In [ ]:
model = carregar_word2vec()
X_train_word2vec, X_val_word2vec, X_test_word2vec = word2vec(train, val, test, 'stems', model)
X_train_oversampling_word2vec, X_val_oversampling_word2vec, X_test_oversampling_word2vec = word2vec(train_oversampling, val_oversampling, test_oversampling, 'stems', model)

X_train_meta1_word2vec, X_val_meta1_word2vec, X_test_meta1_word2vec = word2vec(train_meta1, val_meta1, test_meta1, 'stems', model)

(929606, 300)


## WORD2VEC treinado com os nossos dados

In [19]:
import numpy as np
import optuna
from gensim.models import Word2Vec
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import f1_score

#vetor médio da frase
def vetor_frase_w2v(tokens, model):
    vecs = [model.wv[w] for w in tokens if w in model.wv]
    return np.mean(vecs, axis=0) if vecs else np.zeros(model.vector_size)

# matriz dos embeddings
def prep_dados_w2v(dados, coluna, model):
    return np.vstack([vetor_frase_w2v(tokens, model) for tokens in dados[coluna]])

def objective_w2v(trial, train, val):
    # Hiperparâmetros 
    vector_size = trial.suggest_int("vector_size", 50, 300)
    window = trial.suggest_int("window", 3, 10)
    min_count = trial.suggest_int("min_count", 1, 5)
    sg = trial.suggest_categorical("sg", [0, 1])  # 0 = CBOW, 1 = Skip-gram
    epochs = trial.suggest_int("epochs", 5, 20)
    C = trial.suggest_float("C", 1e-3, 10, log=True)
    use_scaler = trial.suggest_categorical("use_scaler", [True, False])

    # Treina Word2Vec com os parâmetros sugeridos
    sentences = train["tokens"]
    model = Word2Vec(
        sentences=sentences,
        vector_size=vector_size,
        window=window,
        min_count=min_count,
        sg=sg,
        workers=4,
        epochs=epochs
    )

    # Transformar dados nos embeddings médios
    X_train = prep_dados_w2v(train, "tokens", model)
    X_val = prep_dados_w2v(val, "tokens", model)
    y_train = train["classe"].values
    y_val = val["classe"].values

    if use_scaler:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)

    # Modelo de classificação (Logistic Regression)
    clf = LogisticRegression(C=C, max_iter=500)
    clf.fit(X_train, y_train)
    preds = clf.predict(X_val)

    return f1_score(y_val, preds, average="macro")

def word2vec_vec(train, val, test):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_w2v(trial, train, val),
                   n_trials=30, n_jobs=-1, show_progress_bar=False)

    best_params = study.best_params
    print("Melhores parâmetros:", best_params)
    print("Melhor F1:", study.best_value)

    best_model = Word2Vec(
        sentences=train["tokens"],
        vector_size=best_params["vector_size"],
        window=best_params["window"],
        min_count=best_params["min_count"],
        sg=best_params["sg"],
        workers=4,
        epochs=best_params["epochs"]
    )

    # Criar embeddings médios para todos os conjuntos
    X_train = prep_dados_w2v(train, "tokens", best_model)
    X_val = prep_dados_w2v(val, "tokens", best_model)
    X_test = prep_dados_w2v(test, "tokens", best_model)

    if best_params["use_scaler"]:
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)
        X_test = scaler.transform(X_test)

    return X_train, X_val, X_test


In [ ]:
X_train_w2v_trained, X_val_w2v_trained, X_test_w2v_trained = word2vec_vec(train, val, test)
X_train_over_w2v_trained, X_val_over_w2v_trained, X_test_over_w2v_trained= word2vec_vec(train_oversampling, val_oversampling, test_oversampling)

X_train_meta1_w2v_trained, X_val_meta1_w2v_trained, X_test_meta1_w2v_trained = word2vec_vec(train_meta1, val_meta1, test_meta1)

c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 500 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=500).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 500 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=500).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also re

Melhores parâmetros: {'vector_size': 290, 'window': 9, 'min_count': 1, 'sg': 1, 'epochs': 19, 'C': 6.121006668710887, 'use_scaler': True}
Melhor F1: 0.6036808754743623


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 500 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=500).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 500 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=500).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also re

Melhores parâmetros: {'vector_size': 250, 'window': 9, 'min_count': 4, 'sg': 1, 'epochs': 5, 'C': 8.821638367237977, 'use_scaler': False}
Melhor F1: 0.6632855794146116


## Combinar CountVectorizer com TF-IDF

In [21]:
from scipy.sparse import hstack

X_train_freq = hstack([X_train_count_vec, X_train_tfidf_vec])
X_val_freq   = hstack([X_val_count_vec, X_val_tfidf_vec])
X_test_freq  = hstack([X_test_count_vec, X_test_tfidf_vec])

X_train_freq_over = hstack([X_train_oversampling_count_vec, X_train_oversampling_tfidf_vec])
X_val_freq_over   = hstack([X_val_oversampling_count_vec, X_val_oversampling_tfidf_vec])
X_test_freq_over  = hstack([X_test_oversampling_count_vec, X_test_oversampling_tfidf_vec])


In [ ]:
#com o pré-processamento da meta 1
X_train_freq_meta1 = hstack([X_train_meta1_count_vec, X_train_meta1_tfidf_vec])
X_val_freq_meta1   = hstack([X_val_meta1_count_vec, X_val_meta1_tfidf_vec])
X_test_freq_meta1  = hstack([X_test_meta1_count_vec, X_test_meta1_tfidf_vec])

# Aplicar modelos de classificação

## Naive Bayes 

-   Multinomial

In [22]:
def objective_naive_MN(trial, X_train, X_val, y_train, y_val):
    # Hiperparâmetros a otimizar
    alpha = trial.suggest_float("alpha", 1e-3, 2.0, log=True)
    
    # Modelo
    clf = MultinomialNB(alpha=alpha)
    clf.fit(X_train, y_train)
    
    y_val_pred = clf.predict(X_val)
    f1 = f1_score(y_val, y_val_pred, average='weighted')  
    
    return f1

def Naive_Bayes_MN(X_train, X_val, X_test, y_train, y_val, n_trials=30):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_naive_MN(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False)
    
    print("Melhores hiperparâmetros encontrados:")
    print(study.best_params)
    print(f"Melhor F1 (validação): {study.best_value:.4f}")
    
    # Treinar o modelo final com o melhor alpha
    best_model = MultinomialNB(**study.best_params)
    best_model.fit(X_train, y_train)
    
    # Previsão no conjunto de teste
    y_test_pred = best_model.predict(X_test)
    
    return y_test_pred

In [23]:
from sklearn.metrics import f1_score

def objective_naive_GS(trial, X_train, X_val, y_train, y_val):
    # Definir o hiperparâmetro a otimizar
    var_smoothing = trial.suggest_float("var_smoothing", 1e-12, 1e-3, log=True)
    # Criar o modelo
    model = GaussianNB(var_smoothing=var_smoothing)
    # Treinar
    model.fit(X_train, y_train)
    # Avaliar no conjunto de validação
    y_pred = model.predict(X_val)
    f1 = f1_score(y_val, y_pred, average="macro")

    return f1

def naive_bayes_gs(X_train, X_val, X_test, y_train, y_val, n_trials=30):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_naive_GS(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False)
    print("Melhores hiperparâmetros encontrados:")
    print(study.best_params)
    # Treinar o melhor modelo com os melhores parâmetros
    best_model = GaussianNB(**study.best_params)
    best_model.fit(X_train, y_train)
    # Avaliar no conjunto de teste
    y_test_pred = best_model.predict(X_test)

    return y_test_pred

In [24]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_NB = Naive_Bayes_MN(X_train_count_vec,X_val_count_vec,X_test_count_vec,y_train,y_val)
print(metrics.classification_report(y_test, count_vec_predicted_NB))
print(metrics.confusion_matrix(y_test, count_vec_predicted_NB))

print("\ncom oversampling")
count_vec_over_predicted_NB = Naive_Bayes_MN(X_train_oversampling_count_vec,X_val_oversampling_count_vec,X_test_oversampling_count_vec,y_train_oversampling,y_val_oversampling)
print(metrics.classification_report(y_test_oversampling, count_vec_over_predicted_NB))
print(metrics.confusion_matrix(y_test_oversampling, count_vec_over_predicted_NB))

----COUNT VECTORIZER ----
Melhores hiperparâmetros encontrados:
{'alpha': 0.8068820154260331}
Melhor F1 (validação): 0.8099
              precision    recall  f1-score   support

          -1       0.72      0.78      0.75       417
           0       0.85      0.84      0.85      1273
           1       0.32      0.30      0.31       168

    accuracy                           0.78      1858
   macro avg       0.63      0.64      0.64      1858
weighted avg       0.78      0.78      0.78      1858

[[ 327   80   10]
 [ 110 1066   97]
 [  16  101   51]]

com oversampling
Melhores hiperparâmetros encontrados:
{'alpha': 1.7988630398124963}
Melhor F1 (validação): 0.6957
              precision    recall  f1-score   support

          -1       0.77      0.79      0.78      1273
           0       0.55      0.77      0.64      1273
           1       0.71      0.41      0.52      1273

    accuracy                           0.66      3819
   macro avg       0.68      0.66      0.65      381

In [25]:
print ("----TF-IDF ----")
tfidf_vec_predicted_NB= Naive_Bayes_MN(X_train_tfidf_vec, X_val_tfidf_vec, X_test_tfidf_vec, y_train, y_val) 
print(metrics.classification_report(y_test, tfidf_vec_predicted_NB))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_NB))

print("\n com oversampling")
tfidf_vec_oversampling_predicted_NB= Naive_Bayes_MN(X_train_oversampling_tfidf_vec, X_val_oversampling_tfidf_vec, X_test_oversampling_tfidf_vec, y_train_oversampling, y_val_oversampling) 
print(metrics.classification_report(y_test_oversampling, tfidf_vec_oversampling_predicted_NB))
print(metrics.confusion_matrix(y_test_oversampling, tfidf_vec_oversampling_predicted_NB))

----TF-IDF ----
Melhores hiperparâmetros encontrados:
{'alpha': 0.1533667469820008}
Melhor F1 (validação): 0.7562
              precision    recall  f1-score   support

          -1       0.79      0.51      0.62       417
           0       0.77      0.96      0.86      1273
           1       1.00      0.02      0.04       168

    accuracy                           0.78      1858
   macro avg       0.85      0.50      0.50      1858
weighted avg       0.80      0.78      0.73      1858

[[ 212  205    0]
 [  46 1227    0]
 [  10  155    3]]

 com oversampling
Melhores hiperparâmetros encontrados:
{'alpha': 0.011582982512010613}
Melhor F1 (validação): 0.6991
              precision    recall  f1-score   support

          -1       0.76      0.72      0.74      1273
           0       0.52      0.71      0.60      1273
           1       0.61      0.42      0.49      1273

    accuracy                           0.62      3819
   macro avg       0.63      0.62      0.61      3819
weigh

In [26]:
print("count vec + tf idf")
freq_predicted_NB= Naive_Bayes_MN(X_train_freq, X_val_freq, X_test_freq, y_train, y_val, n_trials=30) 
print(metrics.classification_report(y_test,freq_predicted_NB ))
print(metrics.confusion_matrix(y_test, freq_predicted_NB))

print("oversampling")
freq_predicted_NB_over= Naive_Bayes_MN(X_train_freq_over, X_val_freq_over, X_test_freq_over, y_train_oversampling, y_val_oversampling, n_trials=30) 
print(metrics.classification_report(y_test_oversampling,freq_predicted_NB_over))
print(metrics.confusion_matrix(y_test_oversampling, freq_predicted_NB_over))

count vec + tf idf
Melhores hiperparâmetros encontrados:
{'alpha': 1.1604864062521143}
Melhor F1 (validação): 0.8078
              precision    recall  f1-score   support

          -1       0.70      0.78      0.74       417
           0       0.85      0.83      0.84      1273
           1       0.30      0.28      0.29       168

    accuracy                           0.77      1858
   macro avg       0.62      0.63      0.62      1858
weighted avg       0.77      0.77      0.77      1858

[[ 326   80   11]
 [ 120 1055   98]
 [  20  101   47]]
oversampling
Melhores hiperparâmetros encontrados:
{'alpha': 1.9237907920055912}
Melhor F1 (validação): 0.6932
              precision    recall  f1-score   support

          -1       0.76      0.80      0.78      1273
           0       0.56      0.76      0.64      1273
           1       0.69      0.40      0.51      1273

    accuracy                           0.65      3819
   macro avg       0.67      0.65      0.64      3819
weighted a

In [27]:
#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_NB= naive_bayes_gs(X_train_word2vec, X_val_word2vec, X_test_word2vec, y_train, y_val, n_trials=30) 
print(metrics.classification_report(y_test, word2vec_predicted_NB))
print(metrics.confusion_matrix(y_test, word2vec_predicted_NB))

print ("oversampling")
word2vec_over_predicted_NB= naive_bayes_gs(X_train_oversampling_word2vec, X_val_oversampling_word2vec, X_test_oversampling_word2vec, y_train_oversampling, y_val_oversampling, n_trials=30) 
print(metrics.classification_report(y_test_oversampling, word2vec_over_predicted_NB))
print(metrics.confusion_matrix(y_test_oversampling, word2vec_over_predicted_NB))


----WORD2VEC ----
Melhores hiperparâmetros encontrados:
{'var_smoothing': 0.000630712599801614}
              precision    recall  f1-score   support

          -1       0.44      0.42      0.43       417
           0       0.84      0.32      0.47      1273
           1       0.11      0.62      0.18       168

    accuracy                           0.37      1858
   macro avg       0.46      0.46      0.36      1858
weighted avg       0.68      0.37      0.43      1858

[[175  44 198]
 [196 409 668]
 [ 30  33 105]]
oversampling
Melhores hiperparâmetros encontrados:
{'var_smoothing': 1.1308814318189501e-07}
              precision    recall  f1-score   support

          -1       0.57      0.44      0.50      1273
           0       0.57      0.27      0.36      1273
           1       0.39      0.68      0.50      1273

    accuracy                           0.46      3819
   macro avg       0.51      0.46      0.45      3819
weighted avg       0.51      0.46      0.45      3819

[[5

In [28]:
#word2vec treinado com os nossos dados
word2vec_trained_predicted_NB= naive_bayes_gs(X_train_w2v_trained, X_val_w2v_trained, X_test_w2v_trained, y_train, y_val, n_trials=30) 
print(metrics.classification_report(y_test, word2vec_trained_predicted_NB))
print(metrics.confusion_matrix(y_test, word2vec_trained_predicted_NB))

print("oversampling")
word2vec_oversampling_trained_predicted_NB= naive_bayes_gs(X_train_over_w2v_trained, X_val_over_w2v_trained, X_test_over_w2v_trained, y_train_oversampling, y_val_oversampling, n_trials=30) 
print(metrics.classification_report(y_test_oversampling, word2vec_oversampling_trained_predicted_NB))
print(metrics.confusion_matrix(y_test_oversampling, word2vec_oversampling_trained_predicted_NB))

Melhores hiperparâmetros encontrados:
{'var_smoothing': 1.4359994095494946e-11}
              precision    recall  f1-score   support

          -1       0.65      0.63      0.64       417
           0       0.88      0.45      0.60      1273
           1       0.15      0.73      0.25       168

    accuracy                           0.52      1858
   macro avg       0.56      0.60      0.50      1858
weighted avg       0.77      0.52      0.57      1858

[[263  45 109]
 [124 572 577]
 [ 15  31 122]]
oversampling
Melhores hiperparâmetros encontrados:
{'var_smoothing': 2.03688686507296e-09}
              precision    recall  f1-score   support

          -1       0.58      0.75      0.66      1273
           0       0.48      0.66      0.55      1273
           1       0.37      0.13      0.19      1273

    accuracy                           0.51      3819
   macro avg       0.48      0.51      0.47      3819
weighted avg       0.48      0.51      0.47      3819

[[952 199 122]
 [277 

In [29]:
#Glove
print ("----GLOVE ----")
glove_predicted_NB= naive_bayes_gs(X_train_glove, X_val_glove, X_test_glove, y_train, y_val, n_trials=30) 
print(metrics.classification_report(y_test, glove_predicted_NB))
print(metrics.confusion_matrix(y_test, glove_predicted_NB))

----GLOVE ----


NameError: name 'X_train_glove' is not defined

## SVM

In [30]:
def objective(trial, X_train, X_val, y_train, y_val):

    #Definir hiperparâmetros a testar 
    C = trial.suggest_float('C', 0.01, 10.0, log=True)
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
    degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3

    #Criar modelo SVM 
    model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree)
    model = SVC(C=C, kernel=kernel, gamma=gamma, degree=degree)

    #treinar modelo com conjunto de treino
    model.fit(X_train, y_train)

    #prever no conjunto de validação
    y_pred = model.predict(X_val)

    #calcular f1
    f1 = f1_score(y_val, y_pred, average='macro')

    return f1

def svm(X_train, X_val, X_test, y_train, y_val, n_trials=30):
    # Chamar o Optuna para otimizar os parâmetros
    objeto_para_otimizar = optuna.create_study(direction='maximize')
    objeto_para_otimizar.optimize(lambda trial: objective(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False )

    print("Melhores hiperparâmetros encontrados:")
    print(objeto_para_otimizar.best_params)

    # Treinar o melhor modelo
    best_model = SVC(**objeto_para_otimizar.best_params)
    best_model.fit(X_train, y_train)

    #  Avaliar no conjunto de teste
    y_pred_test = best_model.predict(X_test)

    return y_pred_test

In [31]:
print ("----COUNT VECTORIZER ----")
count_vec_predicted_SVM= svm(X_train_count_vec, X_val_count_vec, X_test_count_vec, y_train, y_val, n_trials=30) 
print(metrics.classification_report(
y_test, count_vec_predicted_SVM))
print(metrics.confusion_matrix(y_test, count_vec_predicted_SVM))

print("\ncom oversampling")
count_vec_over_predicted_SVM = svm(X_train_oversampling_count_vec, X_val_oversampling_count_vec, X_test_oversampling_count_vec, y_train_oversampling, y_val_oversampling, n_trials=30)
print(metrics.classification_report(y_test_oversampling, count_vec_over_predicted_SVM))
print(metrics.confusion_matrix(y_test_oversampling, count_vec_over_predicted_SVM))

----COUNT VECTORIZER ----
Melhores hiperparâmetros encontrados:
{'C': 1.155407962843028, 'kernel': 'linear', 'gamma': 'scale'}
              precision    recall  f1-score   support

          -1       0.70      0.72      0.71       417
           0       0.83      0.84      0.84      1273
           1       0.25      0.21      0.23       168

    accuracy                           0.76      1858
   macro avg       0.59      0.59      0.59      1858
weighted avg       0.75      0.76      0.75      1858

[[ 299  100   18]
 [ 113 1070   90]
 [  13  119   36]]

com oversampling
Melhores hiperparâmetros encontrados:
{'C': 0.03408939664161886, 'kernel': 'linear', 'gamma': 'auto'}
              precision    recall  f1-score   support

          -1       0.84      0.73      0.78      1273
           0       0.52      0.83      0.64      1273
           1       0.72      0.38      0.50      1273

    accuracy                           0.65      3819
   macro avg       0.69      0.65      0.64  

In [32]:
print ("---- TF-IDF ----")
tfidf_vec_predicted_SVM= svm(X_train_tfidf_vec, X_val_tfidf_vec, X_test_tfidf_vec, y_train, y_val, n_trials=30) 
print(metrics.classification_report(y_test, tfidf_vec_predicted_SVM))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_SVM))

print ("---- TF-IDF ----")
tfidf_vec_predicted_SVM_oversampling= svm(X_train_oversampling_tfidf_vec, X_val_oversampling_tfidf_vec, X_test_oversampling_tfidf_vec, y_train_oversampling, y_val_oversampling, n_trials=30) 
print(metrics.classification_report(y_test_oversampling, tfidf_vec_predicted_SVM_oversampling))
print(metrics.confusion_matrix(y_test_oversampling, tfidf_vec_predicted_SVM_oversampling))

---- TF-IDF ----
Melhores hiperparâmetros encontrados:
{'C': 8.904928075038061, 'kernel': 'linear', 'gamma': 'auto'}
              precision    recall  f1-score   support

          -1       0.67      0.63      0.65       417
           0       0.80      0.87      0.84      1273
           1       0.27      0.12      0.17       168

    accuracy                           0.75      1858
   macro avg       0.58      0.54      0.55      1858
weighted avg       0.72      0.75      0.73      1858

[[ 262  150    5]
 [ 108 1113   52]
 [  22  125   21]]
---- TF-IDF ----
Melhores hiperparâmetros encontrados:
{'C': 0.22823930939231468, 'kernel': 'linear', 'gamma': 'scale'}
              precision    recall  f1-score   support

          -1       0.81      0.74      0.78      1273
           0       0.54      0.72      0.62      1273
           1       0.63      0.48      0.54      1273

    accuracy                           0.65      3819
   macro avg       0.66      0.65      0.65      3819
w

In [33]:
print("count vec + tf idf")
freq_predicted_svm= svm(X_train_freq, X_val_freq, X_test_freq, y_train, y_val, n_trials=30) 
print(metrics.classification_report(y_test,freq_predicted_svm))
print(metrics.confusion_matrix(y_test, freq_predicted_svm))

print("oversampling")
freq_predicted_svm_over= svm(X_train_freq_over, X_val_freq_over, X_test_freq_over, y_train_oversampling, y_val_oversampling, n_trials=30) 
print(metrics.classification_report(y_test_oversampling,freq_predicted_svm_over))
print(metrics.confusion_matrix(y_test_oversampling, freq_predicted_svm_over))

count vec + tf idf
Melhores hiperparâmetros encontrados:
{'C': 0.2667411350495317, 'kernel': 'linear', 'gamma': 'auto'}
              precision    recall  f1-score   support

          -1       0.81      0.67      0.73       417
           0       0.82      0.91      0.86      1273
           1       0.25      0.14      0.18       168

    accuracy                           0.79      1858
   macro avg       0.62      0.57      0.59      1858
weighted avg       0.76      0.79      0.77      1858

[[ 278  129   10]
 [  55 1158   60]
 [  12  133   23]]
oversampling
Melhores hiperparâmetros encontrados:
{'C': 0.016681982809073317, 'kernel': 'linear', 'gamma': 'scale'}
              precision    recall  f1-score   support

          -1       0.82      0.69      0.75      1273
           0       0.52      0.81      0.63      1273
           1       0.69      0.42      0.52      1273

    accuracy                           0.64      3819
   macro avg       0.68      0.64      0.63      3819
w

In [34]:
print ("----WORD2VEC ----")
word2vec_predicted_SVM= svm(X_train_word2vec, X_val_word2vec, X_test_word2vec, y_train, y_val, n_trials=30) 
print(metrics.classification_report(y_test, word2vec_predicted_SVM))
print(metrics.confusion_matrix(y_test, word2vec_predicted_SVM))

print ("C/ oversampling")
word2vec_predicted_SVM_oversampling= svm(X_train_oversampling_word2vec, X_val_oversampling_word2vec, X_test_oversampling_word2vec, y_train_oversampling, y_val_oversampling, n_trials=30) 
print(metrics.classification_report(y_test_oversampling, word2vec_predicted_SVM_oversampling))
print(metrics.confusion_matrix(y_test_oversampling, word2vec_predicted_SVM_oversampling))

----WORD2VEC ----
Melhores hiperparâmetros encontrados:
{'C': 9.836846123151167, 'kernel': 'rbf', 'gamma': 'scale'}
              precision    recall  f1-score   support

          -1       0.58      0.45      0.51       417
           0       0.77      0.89      0.83      1273
           1       0.33      0.10      0.15       168

    accuracy                           0.72      1858
   macro avg       0.56      0.48      0.49      1858
weighted avg       0.69      0.72      0.69      1858

[[ 188  222    7]
 [ 109 1139   25]
 [  25  127   16]]
C/ oversampling
Melhores hiperparâmetros encontrados:
{'C': 0.6138352787200456, 'kernel': 'poly', 'gamma': 'scale', 'degree': 2}
              precision    recall  f1-score   support

          -1       0.62      0.56      0.59      1273
           0       0.47      0.72      0.57      1273
           1       0.55      0.31      0.40      1273

    accuracy                           0.53      3819
   macro avg       0.55      0.53      0.52    

In [35]:
#word2vec treinado com os nossos dados
word2vec_trained_predicted_svm= svm(X_train_w2v_trained, X_val_w2v_trained, X_test_w2v_trained, y_train, y_val, n_trials=30) 
print(metrics.classification_report(y_test, word2vec_trained_predicted_svm))
print(metrics.confusion_matrix(y_test, word2vec_trained_predicted_svm))

print("oversampling")
word2vec_trained_predicted_svm_over= svm(X_train_over_w2v_trained, X_val_over_w2v_trained, X_test_over_w2v_trained, y_train_oversampling, y_val_oversampling, n_trials=30) 
print(metrics.classification_report(y_test_oversampling, word2vec_trained_predicted_svm_over))
print(metrics.confusion_matrix(y_test_oversampling, word2vec_trained_predicted_svm_over))

Melhores hiperparâmetros encontrados:
{'C': 8.9025480634146, 'kernel': 'rbf', 'gamma': 'auto'}
              precision    recall  f1-score   support

          -1       0.72      0.72      0.72       417
           0       0.83      0.90      0.86      1273
           1       0.25      0.07      0.10       168

    accuracy                           0.79      1858
   macro avg       0.60      0.56      0.56      1858
weighted avg       0.75      0.79      0.76      1858

[[ 302  110    5]
 [  94 1151   28]
 [  23  134   11]]
oversampling
Melhores hiperparâmetros encontrados:
{'C': 0.17975185927753537, 'kernel': 'linear', 'gamma': 'auto'}
              precision    recall  f1-score   support

          -1       0.73      0.73      0.73      1273
           0       0.50      0.72      0.59      1273
           1       0.61      0.35      0.44      1273

    accuracy                           0.60      3819
   macro avg       0.61      0.60      0.59      3819
weighted avg       0.61     

In [ ]:
print ("----GLOVE ----")
glove_predicted_SVM= svm(X_train_glove, X_val_glove, X_test_glove, y_train, y_val, n_trials=30) 
print(metrics.classification_report(y_test, glove_predicted_SVM))
print(metrics.confusion_matrix(y_test, glove_predicted_SVM))

## KNN

In [36]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

def KNN(X_train, y_train, X_val, y_val, X_test):
    k_values = range(1, 200,10)  # Determinar  intervalo de valores a testar para k
    best_k = 1
    best_accuracy = 0
    
    k_list = []
    accuracy_list = [] 

    for k in k_values:

        knn = KNeighborsClassifier(n_neighbors=k)  
        knn.fit(X_train, y_train)

        y_pred_KNN = knn.predict(X_val) 
        accuracy = accuracy_score(y_val, y_pred_KNN)  

        k_list.append(k)
        accuracy_list.append(accuracy)  

        if accuracy > best_accuracy: 
            best_accuracy = accuracy
            best_k = k

    knn = KNeighborsClassifier(n_neighbors=best_k)  
    knn.fit(X_train, y_train)

    y_pred_KNN= knn.predict(X_test) 
    print(f"Best k: {best_k}")

    return y_pred_KNN


In [37]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_KNN= KNN(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
print(metrics.classification_report(y_test, count_vec_predicted_KNN))
print(metrics.confusion_matrix(y_test, count_vec_predicted_KNN))

print ("c/ oversampling")
count_vec_predicted_KNN_oversampling= KNN(X_train_oversampling_count_vec, y_train_oversampling, X_val_oversampling_count_vec, y_val_oversampling, X_test_oversampling_count_vec) 
print(metrics.classification_report(y_test_oversampling, count_vec_predicted_KNN_oversampling))
print(metrics.confusion_matrix(y_test_oversampling, count_vec_predicted_KNN_oversampling))

----COUNT VECTORIZER ----
Best k: 51
              precision    recall  f1-score   support

          -1       0.74      0.60      0.66       417
           0       0.79      0.94      0.86      1273
           1       0.00      0.00      0.00       168

    accuracy                           0.78      1858
   macro avg       0.51      0.51      0.51      1858
weighted avg       0.71      0.78      0.74      1858

[[ 251  166    0]
 [  78 1195    0]
 [  10  158    0]]
c/ oversampling


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Best k: 1
              precision    recall  f1-score   support

          -1       0.65      0.72      0.68      1273
           0       0.45      0.78      0.57      1273
           1       0.65      0.11      0.18      1273

    accuracy                           0.53      3819
   macro avg       0.58      0.53      0.48      3819
weighted avg       0.58      0.53      0.48      3819

[[915 325  33]
 [240 991  42]
 [244 892 137]]


In [38]:
#TF-IDF
print ("----TF-IDF ----")
tfidf_vec_predicted_KNN= KNN(X_train_tfidf_vec, y_train, X_val_tfidf_vec, y_val, X_test_tfidf_vec) 
print(metrics.classification_report(y_test, tfidf_vec_predicted_KNN))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_KNN))

print ("c/oversampling")
tfidf_vec_predicted_KNN_oversampling= KNN(X_train_oversampling_tfidf_vec, y_train_oversampling, X_val_oversampling_tfidf_vec, y_val_oversampling, X_test_oversampling_tfidf_vec) 
print(metrics.classification_report(y_test_oversampling, tfidf_vec_predicted_KNN_oversampling))
print(metrics.confusion_matrix(y_test_oversampling, tfidf_vec_predicted_KNN_oversampling))

----TF-IDF ----
Best k: 181
              precision    recall  f1-score   support

          -1       0.74      0.60      0.67       417
           0       0.79      0.94      0.86      1273
           1       0.00      0.00      0.00       168

    accuracy                           0.78      1858
   macro avg       0.51      0.51      0.51      1858
weighted avg       0.71      0.78      0.74      1858

[[ 251  166    0]
 [  78 1195    0]
 [   8  160    0]]
c/oversampling


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Best k: 171
              precision    recall  f1-score   support

          -1       0.66      0.88      0.75      1273
           0       0.58      0.67      0.62      1273
           1       0.67      0.34      0.45      1273

    accuracy                           0.63      3819
   macro avg       0.64      0.63      0.61      3819
weighted avg       0.64      0.63      0.61      3819

[[1116  100   57]
 [ 259  859  155]
 [ 318  525  430]]


In [39]:
print("count vec + tf idf")
freq_predicted_KNN= KNN(X_train_freq, y_train,X_val_freq,y_val, X_test_freq) 
print(metrics.classification_report(y_test,freq_predicted_KNN ))
print(metrics.confusion_matrix(y_test, freq_predicted_KNN))

print("oversampling")
freq_predicted_KNN_over= KNN(X_train_freq_over, y_train_oversampling, X_val_freq_over, y_val_oversampling, X_test_freq_over) 
print(metrics.classification_report(y_test_oversampling,freq_predicted_KNN_over))
print(metrics.confusion_matrix(y_test_oversampling, freq_predicted_KNN_over))

count vec + tf idf
Best k: 41
              precision    recall  f1-score   support

          -1       0.74      0.60      0.66       417
           0       0.79      0.94      0.86      1273
           1       0.00      0.00      0.00       168

    accuracy                           0.78      1858
   macro avg       0.51      0.51      0.51      1858
weighted avg       0.71      0.78      0.74      1858

[[ 250  167    0]
 [  77 1196    0]
 [   9  159    0]]
oversampling


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


Best k: 11
              precision    recall  f1-score   support

          -1       0.58      0.80      0.67      1273
           0       0.46      0.64      0.54      1273
           1       0.48      0.11      0.18      1273

    accuracy                           0.52      3819
   macro avg       0.51      0.52      0.46      3819
weighted avg       0.51      0.52      0.46      3819

[[1024  212   37]
 [ 352  811  110]
 [ 401  734  138]]


In [40]:
#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_KNN= KNN(X_train_word2vec, y_train, X_val_word2vec, y_val, X_test_word2vec) 
print(metrics.classification_report(y_test, word2vec_predicted_KNN))
print(metrics.confusion_matrix(y_test, word2vec_predicted_KNN))

print ("c/oversampling")
word2vec_predicted_KNN_oversampling= KNN(X_train_oversampling_word2vec, y_train_oversampling, X_val_oversampling_word2vec, y_val_oversampling, X_test_oversampling_word2vec) 
print(metrics.classification_report(y_test_oversampling, word2vec_predicted_KNN_oversampling))
print(metrics.confusion_matrix(y_test_oversampling, word2vec_predicted_KNN_oversampling))

----WORD2VEC ----
Best k: 11
              precision    recall  f1-score   support

          -1       0.62      0.27      0.37       417
           0       0.73      0.95      0.82      1273
           1       0.20      0.01      0.02       168

    accuracy                           0.71      1858
   macro avg       0.51      0.41      0.41      1858
weighted avg       0.65      0.71      0.65      1858

[[ 112  304    1]
 [  56 1210    7]
 [  13  153    2]]
c/oversampling
Best k: 31
              precision    recall  f1-score   support

          -1       0.54      0.47      0.50      1273
           0       0.48      0.39      0.43      1273
           1       0.41      0.53      0.46      1273

    accuracy                           0.46      3819
   macro avg       0.47      0.46      0.46      3819
weighted avg       0.47      0.46      0.46      3819

[[597 202 474]
 [252 496 525]
 [258 335 680]]


In [44]:
#word2vec treinado com os nossos dados
word2vec_trained_predicted_KNN= KNN(X_train_w2v_trained, y_train, X_val_w2v_trained, y_val, X_test_w2v_trained) 
print(metrics.classification_report(y_test, word2vec_trained_predicted_KNN))
print(metrics.confusion_matrix(y_test, word2vec_trained_predicted_KNN))

print("oversampling")
word2vec_trained_predicted_KNN_over= KNN(X_train_over_w2v_trained, y_train_oversampling, X_val_over_w2v_trained, y_val_oversampling,X_test_over_w2v_trained) 
print(metrics.classification_report(y_test_oversampling, word2vec_trained_predicted_KNN_over))
print(metrics.confusion_matrix(y_test_oversampling, word2vec_trained_predicted_KNN_over))

Best k: 21
              precision    recall  f1-score   support

          -1       0.74      0.65      0.69       417
           0       0.81      0.93      0.87      1273
           1       0.36      0.02      0.04       168

    accuracy                           0.79      1858
   macro avg       0.64      0.54      0.53      1858
weighted avg       0.75      0.79      0.75      1858

[[ 273  144    0]
 [  77 1189    7]
 [  21  143    4]]
oversampling
Best k: 51
              precision    recall  f1-score   support

          -1       0.69      0.67      0.68      1273
           0       0.46      0.83      0.59      1273
           1       0.55      0.12      0.20      1273

    accuracy                           0.54      3819
   macro avg       0.57      0.54      0.49      3819
weighted avg       0.57      0.54      0.49      3819

[[ 848  369   56]
 [ 144 1059   70]
 [ 236  883  154]]


In [ ]:
#Glove
print ("----GLOVE ----")
glove_predicted_KNN= KNN(X_train_glove, y_train, X_val_glove, y_val, X_test_glove) 
print(metrics.classification_report(y_test, glove_predicted_KNN))
print(metrics.confusion_matrix(y_test, glove_predicted_KNN))

## Decison Tree

In [45]:
import optuna
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score

def objective_decision_tree(trial, X_train, X_val, y_train, y_val):
    # Hiperparâmetros a otimizar
    criterion = trial.suggest_categorical("criterion", ["entropy", "gini"])
    max_depth = trial.suggest_int("max_depth", 2, 50)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    ccp_alpha = trial.suggest_float("ccp_alpha", 0.0, 0.01)
    
    # Criar o modelo com os parâmetros sugeridos
    model = DecisionTreeClassifier(
        criterion=criterion,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        ccp_alpha=ccp_alpha,
        random_state=42
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    f1 = f1_score(y_val, y_pred, average="macro")

    return f1


def decision_tree(X_train, y_train, X_val, y_val, X_test,n_trials=30):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_decision_tree(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False)

    print("Melhores hiperparâmetros encontrados:")
    print(study.best_params)
    print("Melhor F1 obtido:", study.best_value)

    # Treinar o modelo final com os melhores parâmetros
    best_model = DecisionTreeClassifier(**study.best_params, random_state=42)
    best_model.fit(X_train, y_train)

    # Prever no conjunto de teste
    y_pred_test = best_model.predict(X_test)

    return y_pred_test


In [46]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_DT= decision_tree(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
print(metrics.classification_report(y_test, count_vec_predicted_DT))
print(metrics.confusion_matrix(y_test, count_vec_predicted_DT))

print ("c/oversampling")
count_vec_predicted_DT_oversampling= decision_tree(X_train_oversampling_count_vec, y_train_oversampling, X_val_oversampling_count_vec, y_val_oversampling, X_test_oversampling_count_vec) 
print(metrics.classification_report(y_test_oversampling, count_vec_predicted_DT_oversampling))
print(metrics.confusion_matrix(y_test_oversampling, count_vec_predicted_DT_oversampling))


----COUNT VECTORIZER ----
Melhores hiperparâmetros encontrados:
{'criterion': 'entropy', 'max_depth': 25, 'min_samples_split': 2, 'min_samples_leaf': 3, 'ccp_alpha': 0.0014559206837436035}
Melhor F1 obtido: 0.5507680953501684
              precision    recall  f1-score   support

          -1       0.77      0.62      0.69       417
           0       0.79      0.93      0.86      1273
           1       0.17      0.04      0.06       168

    accuracy                           0.78      1858
   macro avg       0.58      0.53      0.53      1858
weighted avg       0.73      0.78      0.75      1858

[[ 260  154    3]
 [  66 1180   27]
 [  10  152    6]]
c/oversampling
Melhores hiperparâmetros encontrados:
{'criterion': 'entropy', 'max_depth': 19, 'min_samples_split': 9, 'min_samples_leaf': 2, 'ccp_alpha': 0.00474403157282603}
Melhor F1 obtido: 0.610198235699106
              precision    recall  f1-score   support

          -1       0.82      0.68      0.74      1273
           0     

In [47]:
#TF-IDF
print ("----TF-IDF ----")
tfidf_vec_predicted_DT= decision_tree(X_train_tfidf_vec, y_train, X_val_tfidf_vec, y_val, X_test_tfidf_vec) 
print(metrics.classification_report(y_test, tfidf_vec_predicted_DT))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_DT))

print ("c/oversampling")
tfidf_vec_predicted_DT_oversampling= decision_tree(X_train_oversampling_tfidf_vec, y_train_oversampling, X_val_oversampling_tfidf_vec, y_val_oversampling, X_test_oversampling_tfidf_vec) 
print(metrics.classification_report(y_test_oversampling, tfidf_vec_predicted_DT_oversampling))
print(metrics.confusion_matrix(y_test_oversampling, tfidf_vec_predicted_DT_oversampling))


----TF-IDF ----
Melhores hiperparâmetros encontrados:
{'criterion': 'gini', 'max_depth': 33, 'min_samples_split': 14, 'min_samples_leaf': 7, 'ccp_alpha': 2.3592523720135416e-05}
Melhor F1 obtido: 0.5458773225026033
              precision    recall  f1-score   support

          -1       0.72      0.60      0.66       417
           0       0.78      0.90      0.84      1273
           1       0.16      0.04      0.07       168

    accuracy                           0.76      1858
   macro avg       0.55      0.52      0.52      1858
weighted avg       0.71      0.76      0.73      1858

[[ 250  163    4]
 [  88 1151   34]
 [   8  153    7]]
c/oversampling
Melhores hiperparâmetros encontrados:
{'criterion': 'entropy', 'max_depth': 37, 'min_samples_split': 13, 'min_samples_leaf': 7, 'ccp_alpha': 0.006429921761582082}
Melhor F1 obtido: 0.5984361369248031
              precision    recall  f1-score   support

          -1       0.81      0.68      0.74      1273
           0       0.65  

In [48]:
print("count vec + tf idf")
freq_predicted_DT= decision_tree(X_train_freq, y_train,X_val_freq,y_val, X_test_freq, n_trials=30) 
print(metrics.classification_report(y_test,freq_predicted_DT ))
print(metrics.confusion_matrix(y_test, freq_predicted_DT))

print("oversampling")
freq_predicted_DT_over= decision_tree(X_train_freq_over, y_train_oversampling, X_val_freq_over, y_val_oversampling, X_test_freq_over, n_trials=30) 
print(metrics.classification_report(y_test_oversampling,freq_predicted_DT_over))
print(metrics.confusion_matrix(y_test_oversampling, freq_predicted_DT_over))

count vec + tf idf
Melhores hiperparâmetros encontrados:
{'criterion': 'gini', 'max_depth': 13, 'min_samples_split': 16, 'min_samples_leaf': 6, 'ccp_alpha': 4.2733906337860976e-05}
Melhor F1 obtido: 0.5536996336996337
              precision    recall  f1-score   support

          -1       0.78      0.63      0.69       417
           0       0.80      0.94      0.86      1273
           1       0.32      0.04      0.06       168

    accuracy                           0.79      1858
   macro avg       0.63      0.53      0.54      1858
weighted avg       0.75      0.79      0.75      1858

[[ 261  151    5]
 [  67 1198    8]
 [   7  155    6]]
oversampling
Melhores hiperparâmetros encontrados:
{'criterion': 'entropy', 'max_depth': 28, 'min_samples_split': 14, 'min_samples_leaf': 1, 'ccp_alpha': 0.0031623735520961652}
Melhor F1 obtido: 0.5946687279923211
              precision    recall  f1-score   support

          -1       0.79      0.69      0.73      1273
           0       0.51

In [49]:
#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_DT= decision_tree(X_train_word2vec, y_train, X_val_word2vec, y_val, X_test_word2vec) 
print(metrics.classification_report(y_test, word2vec_predicted_DT))
print(metrics.confusion_matrix(y_test, word2vec_predicted_DT))

print ("c/oversampling")
word2vec_predicted_DT_over= decision_tree(X_train_oversampling_word2vec, y_train_oversampling, X_val_oversampling_word2vec, y_val_oversampling, X_test_oversampling_word2vec) 
print(metrics.classification_report(y_test_oversampling, word2vec_predicted_DT_over))
print(metrics.confusion_matrix(y_test_oversampling, word2vec_predicted_DT_over))



----WORD2VEC ----
Melhores hiperparâmetros encontrados:
{'criterion': 'entropy', 'max_depth': 47, 'min_samples_split': 11, 'min_samples_leaf': 1, 'ccp_alpha': 0.0036997943352353414}
Melhor F1 obtido: 0.44793490802052477
              precision    recall  f1-score   support

          -1       0.39      0.31      0.34       417
           0       0.72      0.85      0.78      1273
           1       0.18      0.02      0.04       168

    accuracy                           0.66      1858
   macro avg       0.43      0.40      0.39      1858
weighted avg       0.60      0.66      0.62      1858

[[ 129  285    3]
 [ 173 1085   15]
 [  29  135    4]]
c/oversampling
Melhores hiperparâmetros encontrados:
{'criterion': 'entropy', 'max_depth': 12, 'min_samples_split': 11, 'min_samples_leaf': 5, 'ccp_alpha': 0.003806690003661194}
Melhor F1 obtido: 0.4585068642215681
              precision    recall  f1-score   support

          -1       0.43      0.49      0.46      1273
           0       0

In [50]:
#word2vec treinado com os nossos dados
word2vec_trained_predicted_DT= decision_tree(X_train_w2v_trained, y_train, X_val_w2v_trained, y_val, X_test_w2v_trained, n_trials=30) 
print(metrics.classification_report(y_test, word2vec_trained_predicted_DT))
print(metrics.confusion_matrix(y_test, word2vec_trained_predicted_DT))

print("oversampling")
word2vec_trained_predicted_DT_over= decision_tree(X_train_over_w2v_trained, y_train_oversampling, X_val_over_w2v_trained, y_val_oversampling,X_test_over_w2v_trained, n_trials=30) 
print(metrics.classification_report(y_test_oversampling, word2vec_trained_predicted_DT_over))
print(metrics.confusion_matrix(y_test_oversampling, word2vec_trained_predicted_DT_over))

Melhores hiperparâmetros encontrados:
{'criterion': 'gini', 'max_depth': 34, 'min_samples_split': 14, 'min_samples_leaf': 3, 'ccp_alpha': 0.0001381633030137809}
Melhor F1 obtido: 0.5721000851597866
              precision    recall  f1-score   support

          -1       0.61      0.62      0.62       417
           0       0.81      0.81      0.81      1273
           1       0.19      0.18      0.18       168

    accuracy                           0.71      1858
   macro avg       0.54      0.54      0.54      1858
weighted avg       0.71      0.71      0.71      1858

[[ 260  137   20]
 [ 136 1029  108]
 [  31  107   30]]
oversampling
Melhores hiperparâmetros encontrados:
{'criterion': 'gini', 'max_depth': 23, 'min_samples_split': 16, 'min_samples_leaf': 10, 'ccp_alpha': 0.0027320940944740267}
Melhor F1 obtido: 0.5971706309811452
              precision    recall  f1-score   support

          -1       0.71      0.74      0.73      1273
           0       0.50      0.73      0.60  

In [ ]:
#Glove
print ("---- GLOVE ----")
glove_predicted_DT= decision_tree(X_train_glove, y_train, X_val_glove, y_val, X_test_glove) 
print(metrics.classification_report(y_test, glove_predicted_DT))
print(metrics.confusion_matrix(y_test, glove_predicted_DT))

## Random Florest

In [51]:
import optuna
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

def objective_random_forest(trial, X_train, X_val, y_train, y_val):
    # Hiperparâmetros a otimizar
    n_estimators = trial.suggest_int("n_estimators", 50, 300, step=50)
    max_depth = trial.suggest_int("max_depth", 5, 50)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", None])
    criterion = trial.suggest_categorical("criterion", ["gini", "entropy"])
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)

    # Criar o modelo
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_features=max_features,
        criterion=criterion,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42,
        n_jobs=-1
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    f1 = f1_score(y_val, y_pred, average="macro")

    return f1


def random_florest(X_train, y_train, X_val, y_val, X_test, n_trials=50):
    study = optuna.create_study(direction="maximize")
    study.optimize(lambda trial: objective_random_forest(trial, X_train, X_val, y_train, y_val),
                   n_trials=n_trials, n_jobs=-1, show_progress_bar=False)

    print("Melhores hiperparâmetros Random Forest:")
    print(study.best_params)
    print("Melhor F1 obtido:", study.best_value)

    best_model = RandomForestClassifier(**study.best_params, random_state=42, n_jobs=-1)
    best_model.fit(X_train, y_train)

    y_pred_test = best_model.predict(X_test)

    return y_pred_test


In [52]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_RF= random_florest(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
print(metrics.classification_report(y_test, count_vec_predicted_RF))
print(metrics.confusion_matrix(y_test, count_vec_predicted_RF))

print ("c/oversampling")
count_vec_predicted_RF_over= random_florest(X_train_oversampling_count_vec, y_train_oversampling, X_val_oversampling_count_vec, y_val_oversampling, X_test_oversampling_count_vec) 
print(metrics.classification_report(y_test_oversampling, count_vec_predicted_RF_over))
print(metrics.confusion_matrix(y_test_oversampling, count_vec_predicted_RF_over))

----COUNT VECTORIZER ----
Melhores hiperparâmetros Random Forest:
{'n_estimators': 200, 'max_depth': 36, 'max_features': None, 'criterion': 'gini', 'min_samples_split': 4, 'min_samples_leaf': 2}
Melhor F1 obtido: 0.5710808944215292
              precision    recall  f1-score   support

          -1       0.80      0.66      0.72       417
           0       0.80      0.95      0.87      1273
           1       0.55      0.04      0.07       168

    accuracy                           0.80      1858
   macro avg       0.72      0.55      0.55      1858
weighted avg       0.78      0.80      0.76      1858

[[ 274  142    1]
 [  61 1208    4]
 [   7  155    6]]
c/oversampling
Melhores hiperparâmetros Random Forest:
{'n_estimators': 200, 'max_depth': 43, 'max_features': 'log2', 'criterion': 'gini', 'min_samples_split': 13, 'min_samples_leaf': 5}
Melhor F1 obtido: 0.6905909568779465
              precision    recall  f1-score   support

          -1       0.76      0.79      0.78      1273

In [53]:
#TF-IDF
print ("----TF-IDF ----")
tfidf_vec_predicted_RF= random_florest(X_train_tfidf_vec, y_train, X_val_tfidf_vec, y_val, X_test_tfidf_vec) 
print(metrics.classification_report(y_test, tfidf_vec_predicted_RF))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_RF))

#TF-IDF
print ("c/oversampling")
tfidf_vec_predicted_RF_over= random_florest(X_train_oversampling_tfidf_vec, y_train_oversampling, X_val_oversampling_tfidf_vec, y_val_oversampling, X_test_oversampling_tfidf_vec) 
print(metrics.classification_report(y_test_oversampling, tfidf_vec_predicted_RF_over))
print(metrics.confusion_matrix(y_test_oversampling, tfidf_vec_predicted_RF_over))

----TF-IDF ----
Melhores hiperparâmetros Random Forest:
{'n_estimators': 150, 'max_depth': 43, 'max_features': 'sqrt', 'criterion': 'entropy', 'min_samples_split': 20, 'min_samples_leaf': 1}
Melhor F1 obtido: 0.5385620509495447
              precision    recall  f1-score   support

          -1       0.78      0.64      0.70       417
           0       0.80      0.95      0.87      1273
           1       0.00      0.00      0.00       168

    accuracy                           0.79      1858
   macro avg       0.53      0.53      0.52      1858
weighted avg       0.72      0.79      0.75      1858

[[ 265  152    0]
 [  63 1209    1]
 [  10  158    0]]
c/oversampling
Melhores hiperparâmetros Random Forest:
{'n_estimators': 200, 'max_depth': 34, 'max_features': 'log2', 'criterion': 'entropy', 'min_samples_split': 17, 'min_samples_leaf': 4}
Melhor F1 obtido: 0.6906231760667535
              precision    recall  f1-score   support

          -1       0.77      0.76      0.76      1273


In [54]:
print("count vec + tf idf")
freq_predicted_RF= random_florest(X_train_freq, y_train,X_val_freq,y_val, X_test_freq, n_trials=30) 
print(metrics.classification_report(y_test,freq_predicted_RF))
print(metrics.confusion_matrix(y_test, freq_predicted_RF))

print("oversampling")
freq_predicted_RF_over= random_florest(X_train_freq_over, y_train_oversampling, X_val_freq_over, y_val_oversampling, X_test_freq_over, n_trials=30) 
print(metrics.classification_report(y_test_oversampling,freq_predicted_RF_over))
print(metrics.confusion_matrix(y_test_oversampling, freq_predicted_RF_over))

count vec + tf idf
Melhores hiperparâmetros Random Forest:
{'n_estimators': 150, 'max_depth': 37, 'max_features': None, 'criterion': 'entropy', 'min_samples_split': 6, 'min_samples_leaf': 1}
Melhor F1 obtido: 0.560136606322754


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


              precision    recall  f1-score   support

          -1       0.80      0.64      0.71       417
           0       0.80      0.95      0.87      1273
           1       0.00      0.00      0.00       168

    accuracy                           0.80      1858
   macro avg       0.53      0.53      0.53      1858
weighted avg       0.72      0.80      0.75      1858

[[ 268  149    0]
 [  62 1211    0]
 [   6  162    0]]
oversampling
Melhores hiperparâmetros Random Forest:
{'n_estimators': 250, 'max_depth': 40, 'max_features': 'log2', 'criterion': 'gini', 'min_samples_split': 7, 'min_samples_leaf': 6}
Melhor F1 obtido: 0.6986316573908139
              precision    recall  f1-score   support

          -1       0.78      0.80      0.79      1273
           0       0.54      0.79      0.64      1273
           1       0.73      0.36      0.49      1273

    accuracy                           0.65      3819
   macro avg       0.68      0.65      0.64      3819
weighted avg     

In [55]:
#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_RF= random_florest(X_train_word2vec, y_train, X_val_word2vec, y_val, X_test_word2vec) 
print(metrics.classification_report(y_test, word2vec_predicted_RF))
print(metrics.confusion_matrix(y_test, word2vec_predicted_RF))

print ("c/oversampling")
word2vec_predicted_RF_over= random_florest(X_train_oversampling_word2vec, y_train_oversampling, X_val_oversampling_word2vec, y_val_oversampling, X_test_oversampling_word2vec) 
print(metrics.classification_report(y_test_oversampling, word2vec_predicted_RF_over))
print(metrics.confusion_matrix(y_test_oversampling, word2vec_predicted_RF_over))

----WORD2VEC ----


KeyboardInterrupt: 

In [ ]:
#word2vec treinado com os nossos dados
word2vec_trained_predicted_RF= random_florest(X_train_w2v_trained, y_train, X_val_w2v_trained, y_val, X_test_w2v_trained, n_trials=30) 
print(metrics.classification_report(y_test, word2vec_trained_predicted_RF))
print(metrics.confusion_matrix(y_test, word2vec_trained_predicted_RF))

print("oversampling")
word2vec_trained_predicted_RF_over= svm(X_train_over_w2v_trained, y_train_oversampling, X_val_over_w2v_trained, y_val_oversampling,X_test_over_w2v_trained, n_trials=30) 
print(metrics.classification_report(y_test_oversampling, word2vec_trained_predicted_RF_over))
print(metrics.confusion_matrix(y_test_oversampling, word2vec_trained_predicted_RF_over))

In [ ]:
#Glove
print ("---- GLOVE ----")
glove_predicted_DT= random_florest(X_train_glove, y_train, X_val_glove, y_val, X_test_glove, y_test) 
print(metrics.classification_report(y_test, glove_predicted_DT))
print(metrics.confusion_matrix(y_test, glove_predicted_DT))

## Neural network

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
import optuna

def objective(X_train, y_train, X_val, y_val, trial):
    hidden_layer_sizes = trial.suggest_categorical('hidden_layer_sizes', [50, 100, 200])
    activation = trial.suggest_categorical('activation', ['relu', 'tanh'])
    learning_rate_init = trial.suggest_float('learning_rate_init', 1e-4, 1e-2, log=True)

    clf = MLPClassifier(
        hidden_layer_sizes=hidden_layer_sizes,
        activation=activation,
        learning_rate_init=learning_rate_init,
        max_iter=300,
        random_state=42
    )

    clf.fit(X_train, y_train)
    preds = clf.predict(X_val)
    f1 = f1_score(y_val, preds, average='macro')
    return f1

def neural_network(X_train, y_train, X_val, y_val, X_test):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(X_train, y_train, X_val, y_val, trial), n_trials=30)
    print("Melhores parâmetros:", study.best_params)

    best_params = study.best_params
    best_model = MLPClassifier(
        hidden_layer_sizes=best_params['hidden_layer_sizes'],
        activation=best_params['activation'],
        learning_rate_init=best_params['learning_rate_init'],
        max_iter=300,
        random_state=42
    )

    best_model.fit(X_train, y_train)
    previsoes = best_model.predict(X_test)
    
    return previsoes


In [ ]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_NN= neural_network(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
print(metrics.classification_report(y_test, count_vec_predicted_NN))
print(metrics.confusion_matrix(y_test, count_vec_predicted_NN))

print ("c/oversampling")
count_vec_predicted_NN_over= neural_network(X_train_oversampling_count_vec, y_train_oversampling, X_val_oversampling_count_vec, y_val_oversampling, X_test_oversampling_count_vec) 
print(metrics.classification_report(y_test_oversampling, count_vec_predicted_NN_over))
print(metrics.confusion_matrix(y_test_oversampling, count_vec_predicted_NN_over))

----COUNT VECTORIZER ----


NameError: name 'X_train_count_vec' is not defined

In [ ]:
#TF-IDF
print ("----TF-IDF ----")
tfidf_vec_predicted_NN= neural_network(X_train_tfidf_vec, y_train, X_val_tfidf_vec, y_val, X_test_tfidf_vec) 
print(metrics.classification_report(y_test, tfidf_vec_predicted_NN))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_NN))

#TF-IDF
print ("c/oversampling")
tfidf_vec_predicted_NN_over= neural_network(X_train_oversampling_tfidf_vec, y_train_oversampling, X_val_oversampling_tfidf_vec, y_val_oversampling, X_test_oversampling_tfidf_vec) 
print(metrics.classification_report(y_test_oversampling, tfidf_vec_predicted_NN_over))
print(metrics.confusion_matrix(y_test_oversampling, tfidf_vec_predicted_NN_over))

In [ ]:
print("count vec + tf idf")
freq_predicted_NN= neural_network(X_train_freq, y_train,X_val_freq,y_val, X_test_freq) 
print(metrics.classification_report(y_test,freq_predicted_NN))
print(metrics.confusion_matrix(y_test, freq_predicted_NN))

print("oversampling")
freq_predicted_NN_over= neural_network(X_train_freq_over, y_train_oversampling, X_val_freq_over, y_val_oversampling, X_test_freq_over) 
print(metrics.classification_report(y_test_oversampling,freq_predicted_NN_over))
print(metrics.confusion_matrix(y_test_oversampling, freq_predicted_NN_over))

count vec + tf idf


c:\Users\didia\anaconda3\Lib\site-packages\optuna\distributions.py:518: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (50,) which is of type tuple.
  warnings.warn(message)
c:\Users\didia\anaconda3\Lib\site-packages\optuna\distributions.py:518: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100,) which is of type tuple.
  warnings.warn(message)
c:\Users\didia\anaconda3\Lib\site-packages\optuna\distributions.py:518: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100, 50) which is of type tuple.
  warnings.warn(message)
c:\Users\didia\anaconda3\Lib\site-packages\optuna\distributions.py:518: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent sto

Melhores parâmetros: {'hidden_layer_sizes': (100, 50), 'activation': 'relu', 'learning_rate_init': 0.005324643450656601}
              precision    recall  f1-score   support

          -1       0.77      0.74      0.75       417
           0       0.84      0.88      0.86      1273
           1       0.25      0.17      0.20       168

    accuracy                           0.78      1858
   macro avg       0.62      0.60      0.60      1858
weighted avg       0.77      0.78      0.77      1858

[[ 308   97   12]
 [  77 1120   76]
 [  15  124   29]]
oversampling


c:\Users\didia\anaconda3\Lib\site-packages\optuna\distributions.py:518: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (50,) which is of type tuple.
  warnings.warn(message)
c:\Users\didia\anaconda3\Lib\site-packages\optuna\distributions.py:518: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100,) which is of type tuple.
  warnings.warn(message)
c:\Users\didia\anaconda3\Lib\site-packages\optuna\distributions.py:518: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent storage but contains (100, 50) which is of type tuple.
  warnings.warn(message)
c:\Users\didia\anaconda3\Lib\site-packages\optuna\distributions.py:518: UserWarning: Choices for a categorical distribution should be a tuple of None, bool, int, float and str for persistent sto

In [ ]:
#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_NN= neural_network(X_train_word2vec, y_train, X_val_word2vec, y_val, X_test_word2vec) 
print(metrics.classification_report(y_test, word2vec_predicted_NN))
print(metrics.confusion_matrix(y_test, word2vec_predicted_NN))

print ("c/oversampling")
word2vec_predicted_NN_over= neural_network(X_train_oversampling_word2vec, y_train_oversampling, X_val_oversampling_word2vec, y_val_oversampling, X_test_oversampling_word2vec) 
print(metrics.classification_report(y_test_oversampling, word2vec_predicted_NN_over))
print(metrics.confusion_matrix(y_test_oversampling, word2vec_predicted_NN_over))


In [ ]:
#word2vec treinado com os nossos dados
word2vec_trained_predicted_NN= neural_network(X_train_w2v_trained, y_train, X_val_w2v_trained, y_val, X_test_w2v_trained, n_trials=30) 
print(metrics.classification_report(y_test, word2vec_trained_predicted_NN))
print(metrics.confusion_matrix(y_test, word2vec_trained_predicted_NN))

print("oversampling")
word2vec_trained_predicted_NN_over= svm(X_train_over_w2v_trained, y_train_oversampling, X_val_over_w2v_trained, y_val_oversampling,X_test_over_w2v_trained, n_trials=30) 
print(metrics.classification_report(y_test_oversampling, word2vec_trained_predicted_NN_over))
print(metrics.confusion_matrix(y_test_oversampling, word2vec_trained_predicted_NN_over))

In [ ]:
#Glove
print ("----GLOVE ----")
glove_predicted_NN= neural_network(X_train_glove, y_train, X_val_glove, y_val, X_test_glove) 
print(metrics.classification_report(y_test, glove_predicted_NN))
print(metrics.confusion_matrix(y_test, glove_predicted_NN))

## Regressão Logística

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
import optuna

def objective(X_train, y_train, X_val, y_val, trial):
    # Hiperparâmetros a otimizar
    C = trial.suggest_float('C', 1e-3, 1e3, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
    solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])  # compatíveis com L1 e L2
    max_iter = trial.suggest_int('max_iter', 100, 1000)

    model = LogisticRegression(
        C=C,
        penalty=penalty,
        solver=solver,
        max_iter=max_iter,
        random_state=42
    )

    model.fit(X_train, y_train)
    preds = model.predict(X_val)

    f1 = f1_score(y_val, preds, average='macro')
    return f1


def logistic_regression(X_train, y_train, X_val, y_val, X_test):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(X_train, y_train, X_val, y_val, trial), n_trials=30)

    print("Melhores parâmetros encontrados:")
    print(study.best_params)

    best_params = study.best_params
    best_model = LogisticRegression(
        **best_params,
        random_state=42
    )

    best_model.fit(X_train, y_train)

    previsoes = best_model.predict(X_test)

    return previsoes


In [ ]:
# Count Vectorizer
print ("----COUNT VECTORIZER ----")
count_vec_predicted_LR= logistic_regression(X_train_count_vec, y_train, X_val_count_vec, y_val, X_test_count_vec) 
print(metrics.classification_report(y_test, count_vec_predicted_LR))
print(metrics.confusion_matrix(y_test, count_vec_predicted_LR))

print ("c/oversampling")
count_vec_predicted_LR_over= logistic_regression(X_train_oversampling_count_vec, y_train_oversampling, X_val_oversampling_count_vec, y_val_oversampling, X_test_oversampling_count_vec) 
print(metrics.classification_report(y_test_oversampling, count_vec_predicted_LR_over))
print(metrics.confusion_matrix(y_test_oversampling, count_vec_predicted_LR_over))

[I 2025-11-13 01:38:47,774] A new study created in memory with name: no-name-f804a6c1-a9ec-4956-92ee-a72dd1f88037
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
[I 2025-11-13 01:38:47,815] Trial 0 finished with value: 0.5759647633129582 and parameters: {'C': 0.5702975431962373, 'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 434}. Best is trial 0 with value: 0.5759647633129582.
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator i

----COUNT VECTORIZER ----


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2025-11-13 01:38:48,033] Trial 2 finished with value: 0.6556576016190188 and parameters: {'C': 100.62436324687397, 'penalty': 'l2', 'solver': 'saga', 'max_iter': 135}. Best is trial 2 with value: 0.6556576016190188.
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2025-11-13 01:38:48,752] Trial 3 finished with value: 0.6504041189506938 and parameters: {'C': 162.82595003814305, 'penalty': 'l2', 'solver': 'saga', 'max_iter': 567}. Best is trial 2 with value: 0.6556576016190188.
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2025-11-13 01:38:49,360] 

Melhores parâmetros encontrados:
{'C': 3.4457245926565587, 'penalty': 'l1', 'solver': 'saga', 'max_iter': 277}


c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2025-11-13 01:43:34,711] A new study created in memory with name: no-name-8b78f7ef-34bf-4cd4-ae02-5e635fa2a503
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
[I 2025-11-13 01:43:34,881] Trial 0 finished with value: 0.677824372328581 and parameters: {'C': 1.6684911551814994, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 894}. Best is trial 0 with value: 0.677824372328581.


              precision    recall  f1-score   support

          -1       0.77      0.64      0.70       417
           0       0.81      0.91      0.86      1273
           1       0.34      0.18      0.24       168

    accuracy                           0.78      1858
   macro avg       0.64      0.58      0.60      1858
weighted avg       0.76      0.78      0.77      1858

[[ 268  139   10]
 [  67 1159   47]
 [  13  125   30]]
c/oversampling


[I 2025-11-13 01:43:36,823] Trial 1 finished with value: 0.7002086643251745 and parameters: {'C': 0.01636145823939285, 'penalty': 'l2', 'solver': 'saga', 'max_iter': 969}. Best is trial 1 with value: 0.7002086643251745.
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
[I 2025-11-13 01:43:37,006] Trial 2 finished with value: 0.6204439095503163 and parameters: {'C': 41.268624677841466, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 568}. Best is trial 1 with value: 0.7002086643251745.
c:\Users\didia\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error wi

Melhores parâmetros encontrados:
{'C': 0.1084583216913827, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 263}
              precision    recall  f1-score   support

          -1       0.84      0.77      0.80      1273
           0       0.54      0.83      0.65      1273
           1       0.72      0.40      0.51      1273

    accuracy                           0.67      3819
   macro avg       0.70      0.67      0.66      3819
weighted avg       0.70      0.67      0.66      3819

[[ 979  219   75]
 [  96 1058  119]
 [  89  681  503]]


In [ ]:
#TF-IDF
print ("----TF-IDF ----")
tfidf_vec_predicted_LR= logistic_regression(X_train_tfidf_vec, y_train, X_val_tfidf_vec, y_val, X_test_tfidf_vec) 
print(metrics.classification_report(y_test, tfidf_vec_predicted_LR))
print(metrics.confusion_matrix(y_test, tfidf_vec_predicted_LR))

#TF-IDF
print ("c/oversampling")
tfidf_vec_predicted_LR_over= logistic_regression(X_train_oversampling_tfidf_vec, y_train_oversampling, X_val_oversampling_tfidf_vec, y_val_oversampling, X_test_oversampling_tfidf_vec) 
print(metrics.classification_report(y_test_oversampling, tfidf_vec_predicted_LR_over))
print(metrics.confusion_matrix(y_test_oversampling, tfidf_vec_predicted_LR_over))



In [ ]:
print("count vec + tf idf")
freq_predicted_LR= logistic_regression(X_train_freq, y_train,X_val_freq,y_val, X_test_freq) 
print(metrics.classification_report(y_test,freq_predicted_RF))
print(metrics.confusion_matrix(y_test, freq_predicted_RF))

print("oversampling")
freq_predicted_LR_over= logistic_regression(X_train_freq_over, y_train_oversampling, X_val_freq_over, y_val_oversampling, X_test_freq_over) 
print(metrics.classification_report(y_test_oversampling,freq_predicted_LR_over))
print(metrics.confusion_matrix(y_test_oversampling, freq_predicted_LR_over))

In [ ]:
#Word2vec
print ("----WORD2VEC ----")
word2vec_predicted_LR= logistic_regression(X_train_word2vec, y_train, X_val_word2vec, y_val, X_test_word2vec) 
print(metrics.classification_report(y_test, word2vec_predicted_LR))
print(metrics.confusion_matrix(y_test, word2vec_predicted_LR))

print ("c/oversampling")
word2vec_predicted_LR_over= logistic_regression(X_train_oversampling_word2vec, y_train_oversampling, X_val_oversampling_word2vec, y_val_oversampling, X_test_oversampling_word2vec) 
print(metrics.classification_report(y_test_oversampling, word2vec_predicted_LR_over))
print(metrics.confusion_matrix(y_test_oversampling, word2vec_predicted_LR_over))

In [ ]:
#word2vec treinado com os nossos dados
word2vec_trained_predicted_LR= logistic_regression(X_train_w2v_trained, y_train, X_val_w2v_trained, y_val, X_test_w2v_trained) 
print(metrics.classification_report(y_test, word2vec_trained_predicted_LR))
print(metrics.confusion_matrix(y_test, word2vec_trained_predicted_LR))

print("oversampling")
word2vec_trained_predicted_LR_over= logistic_regression(X_train_over_w2v_trained, y_train_oversampling, X_val_over_w2v_trained, y_val_oversampling,X_test_over_w2v_trained) 
print(metrics.classification_report(y_test_oversampling, word2vec_trained_predicted_LR_over))
print(metrics.confusion_matrix(y_test_oversampling, word2vec_trained_predicted_LR_over))

In [ ]:
#Glove
print ("----GLOVE ----")
glove_predicted_LR= logistic_regression(X_train_glove, y_train, X_val_glove, y_val, X_test_glove, y_test) 
print(metrics.classification_report(y_test, glove_predicted_LR))
print(metrics.confusion_matrix(y_test, glove_predicted_LR))